In [25]:
from bs4 import BeautifulSoup as bs
import mechanicalsoup
import pandas as pd
import os

# url for phobius
url = "https://phobius.sbc.su.se/index.html"

# read in fasta file as text
single_protein_fasta = open("../data/Proteins/example_protein.fasta", "r").read()
ten_protein_fasta = open("../data/Proteins/ten_tslil_protein_strings.fasta", "r").read()
all_proteins_fasta = open("../data/Proteins/tslil_protein_strings.fasta", "r").read()
print(all_proteins_fasta)

>YMR256C  | Saccharomyces cerevisiae S288C | cytochrome c oxidase subunit VII | protein | length=60
MANKVIQLQKIFQSSTKPLWWRHPRSALYLYPFYAIFAVAVVTPLLYIPNAIRGIKAKKA
>YEL064C  | Saccharomyces cerevisiae S288C | Avt2p | protein | length=60
MSELGEYSKLENKELRTEFELTNFPFPGTTDNDSDDGSQGQNSLNIITPDMDDTLVNDVL
>YPR156C  | Saccharomyces cerevisiae S288C | spermine transporter | protein | length=60
MNRQESINSFNSDETSSLSDVESQQPQQYIPSESGSKSNMAPNQLKLTRTETVKSLQDMG
>YOR003W  | Saccharomyces cerevisiae S288C | putative subtilisin-like protease YSP3 | protein | length=60
MKFSTILPILWANCCLCMIIPDFDGIVRFIENIDGTRSVRAGEGLGQHDPGNFHTEHQHV
>YER087C-B  | Saccharomyces cerevisiae S288C | Arf family guanine nucleotide exchange factor SBH1 | protein | length=60
MSSPTPPGGQRTLQKRKQGSSQKVAASAPKKNTNSNNSILKIYSDEATGLRVDPLVVLFL
>YAR066W  | Saccharomyces cerevisiae S288C | hypothetical protein | protein | length=60
MFNRFNKFQAAVALALLSRGALGDSYTNSTSSADLSSITSVSSASASATASDSLSSSDGT
>YHR215W  | Saccharomyces cerevisiae S288C | acid phosphata

In [10]:
#instantiate browser, open url, select form
browser = mechanicalsoup.StatefulBrowser(soup_config={'features': 'html.parser'})
browser.open(url)
browser.select_form('form')

browser.form.print_summary()

<textarea cols="80" name="protseq" rows="6"></textarea>
<input name="protfile" size="15" style="font-family: helvetica,arial,sans-serif;" type="file"/>
<input name="format" type="radio" value="short"/>
<input name="format" type="radio" value="nog"/>
<input checked="checked" name="format" type="radio" value="plp"/>
<input type="submit"/>
<input type="reset"/>


In [11]:
# fill in form and check
#browser["protseq"] = all_proteins_fasta
browser["format"] = "nog"
browser.form.set("protfile", "../data/Proteins/ten_tslil_protein_strings.fasta")
browser.form.print_summary()
# browser.launch_browser()

<textarea cols="80" name="protseq" rows="6"></textarea>
<input name="protfile" size="15" style="font-family: helvetica,arial,sans-serif;" type="file" value="../data/Proteins/ten_tslil_protein_strings.fasta"/>
<input name="format" type="radio" value="short"/>
<input checked="" name="format" type="radio" value="nog"/>
<input name="format" type="radio" value="plp"/>
<input type="submit"/>
<input type="reset"/>


In [12]:
response = browser.submit_selected()
print(response.text)

<!DOCTYPE html
	PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
	 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en-US" xml:lang="en-US">
<head>
<title>Phobius prediction</title>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />
</head>
<body>
<h2>Phobius prediction</h2><hr /><pre>ID   YMR256C
FT   TOPO_DOM      1     27       NON CYTOPLASMIC.
FT   TRANSMEM     28     48       
FT   TOPO_DOM     49     60       CYTOPLASMIC.
//
ID   YEL064C
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YPR156C
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YOR003W
FT   SIGNAL        1     17       
FT   REGION        1      2       N-REGION.
FT   REGION        3     12       H-REGION.
FT   REGION       13     17       C-REGION.
FT   TOPO_DOM     18     60       NON CYTOPLASMIC.
//
ID   YER087C-B
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YAR066W
FT   SIGNAL        1     23      

In [13]:
# parse response to get all data in <pre> tag
soup = bs(response.text, 'html.parser')
out = soup.find('pre')
print(out)

<pre>ID   YMR256C
FT   TOPO_DOM      1     27       NON CYTOPLASMIC.
FT   TRANSMEM     28     48       
FT   TOPO_DOM     49     60       CYTOPLASMIC.
//
ID   YEL064C
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YPR156C
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YOR003W
FT   SIGNAL        1     17       
FT   REGION        1      2       N-REGION.
FT   REGION        3     12       H-REGION.
FT   REGION       13     17       C-REGION.
FT   TOPO_DOM     18     60       NON CYTOPLASMIC.
//
ID   YER087C-B
FT   TOPO_DOM      1     60       NON CYTOPLASMIC.
//
ID   YAR066W
FT   SIGNAL        1     23       
FT   REGION        1      7       N-REGION.
FT   REGION        8     18       H-REGION.
FT   REGION       19     23       C-REGION.
FT   TOPO_DOM     24     60       NON CYTOPLASMIC.
//
ID   YHR215W
FT   SIGNAL        1     17       
FT   REGION        1      3       N-REGION.
FT   REGION        4     12       H-REGION.
FT   REGION       13     17       C-REG

In [14]:
# right -> left:
# splits output text into list of protein outputs 
# then splits each protein output into list of non-empty lines and collapses whitespace
out_split = [[y.split() for y in x.split("\n") if y != ''] for x in out.text.split("//")][:-1]
out_split

[[['ID', 'YMR256C'],
  ['FT', 'TOPO_DOM', '1', '27', 'NON', 'CYTOPLASMIC.'],
  ['FT', 'TRANSMEM', '28', '48'],
  ['FT', 'TOPO_DOM', '49', '60', 'CYTOPLASMIC.']],
 [['ID', 'YEL064C'], ['FT', 'TOPO_DOM', '1', '60', 'NON', 'CYTOPLASMIC.']],
 [['ID', 'YPR156C'], ['FT', 'TOPO_DOM', '1', '60', 'NON', 'CYTOPLASMIC.']],
 [['ID', 'YOR003W'],
  ['FT', 'SIGNAL', '1', '17'],
  ['FT', 'REGION', '1', '2', 'N-REGION.'],
  ['FT', 'REGION', '3', '12', 'H-REGION.'],
  ['FT', 'REGION', '13', '17', 'C-REGION.'],
  ['FT', 'TOPO_DOM', '18', '60', 'NON', 'CYTOPLASMIC.']],
 [['ID', 'YER087C-B'], ['FT', 'TOPO_DOM', '1', '60', 'NON', 'CYTOPLASMIC.']],
 [['ID', 'YAR066W'],
  ['FT', 'SIGNAL', '1', '23'],
  ['FT', 'REGION', '1', '7', 'N-REGION.'],
  ['FT', 'REGION', '8', '18', 'H-REGION.'],
  ['FT', 'REGION', '19', '23', 'C-REGION.'],
  ['FT', 'TOPO_DOM', '24', '60', 'NON', 'CYTOPLASMIC.']],
 [['ID', 'YHR215W'],
  ['FT', 'SIGNAL', '1', '17'],
  ['FT', 'REGION', '1', '3', 'N-REGION.'],
  ['FT', 'REGION', '4', '12',

In [15]:
assert len(out_split) == len(ten_protein_fasta.split(">")) - 1, "number of proteins in output does not match number of proteins in input"

predicted = []
unpredicted = []
for protein in out_split:
    if 'TRANSMEM' in [x[1] for x in protein]:
        predicted.append(protein)
    else:
        unpredicted.append(protein)

window_df = pd.DataFrame(columns = ["ID", "start", "end"])
for protein in predicted:
    # extract category column
    col_2 = [x[1] for x in protein]

    # extract protein ID, and start and end of transmembrane region
    proteinID = col_2[0]
    transmem_row = col_2.index("TRANSMEM")
    start = protein[transmem_row][2]
    end = protein[transmem_row][3]
    
    # append to dataframe
    new_row = pd.DataFrame([[proteinID, start, end]], columns = ["ID", "start", "end"])
    window_df = pd.concat([window_df, new_row], ignore_index=True)

window_df

,ID,start,end
0,YMR256C,28,48
1,YDR329C,18,39


In [43]:
def phobius(path: str) -> pd.DataFrame:
    fileString = open(path, "r").read()

    #instantiate browser, open url, select form
    browser = mechanicalsoup.StatefulBrowser(soup_config={'features': 'html.parser'})
    browser.open(url)
    browser.select_form('form')

    # fill in form and check
    browser["format"] = "nog"
    browser.form.set("protfile", path)

    response = browser.submit_selected()

    # parse response to get all data in <pre> tag
    soup = bs(response.text, 'html.parser')
    out = soup.find('pre')

    # right -> left:
    # splits output text into list of protein outputs 
    # then splits each protein output into list of non-empty lines and collapses whitespace
    out_split = [[y.split() for y in x.split("\n") if y != ''] for x in out.text.split("//")][:-1]

    assert len(out_split) == len(fileString.split(">")) - 1, "number of proteins in output does not match number of proteins in input"

    # split into predicted and unpredicted
    predicted = []
    unpredicted = []
    for protein in out_split:
        if 'TRANSMEM' in [x[1] for x in protein]:
            predicted.append(protein)
        else:
            unpredicted.append(protein)

    # populate dataframe with predicted transmembrane window sizes
    window_df = pd.DataFrame(columns = ["ID", "start", "end"], dtype=int)
    for protein in predicted:
        # extract category column
        col_2 = [x[1] for x in protein]

        # extract protein ID, and start and end of transmembrane region
        proteinID = col_2[0]
        transmem_row = col_2.index("TRANSMEM")
        start = int(protein[transmem_row][2])
        end = int(protein[transmem_row][3])
        
        # append to dataframe
        new_row = pd.DataFrame([[proteinID, start, end]], columns = ["ID", "start", "end"])
        window_df = pd.concat([window_df, new_row], ignore_index=True)


    return window_df

phobius("../data/Proteins/ten_tslil_protein_strings.fasta").dtypes

ID       object
start     int64
end       int64
dtype: object